In [166]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyClassifier
from sklearn.metrics import roc_auc_score

%run ../src/Munger.py
%run ../src/Models.py
%run ../src/utils.py

In [6]:
# load dataset
numer_ai = pd.read_csv('../data/numerai_datasets/numerai_training_data.csv')
test = pd.read_csv('../data/numerai_datasets/numerai_tournament_data.csv')

In [7]:
# split into training and validation sets as per the competition instructions
train = numer_ai[numer_ai.validation == 0]
validation = numer_ai[numer_ai.validation == 1]

In [122]:
munger = Munger(train, validation, test)

In [123]:
## remove correlated features
munger.remove_correlated_features()

In [124]:
X_train = munger.X
y_train = munger.y

X_validation = munger.X_validation
y_validation = munger.y_validation

X_test = munger.test

In [167]:
model = Models()
clf = model.random_forest_classifier()

In [168]:
# fit a model
clf.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=5,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [ ]:
## 
from sklearn.cross_validation import train_test_split

X_val, X_test, y_val, y_test = train_test_split(X_validation, y_validation, 
                                                test_size=0.33, random_state=44)

In [169]:
## predictions
predsTrain = clf.predict_proba(X_train)[:, 1]
predsValidation = clf.predict_proba(X_validation)[:, 1]

In [170]:
## check to see how this dummy classifier performs
print 'ROC AUC Score on the training examples %f ' %(roc_auc_score(y_train, predsTrain))
print 'ROC AUC Score on the validation examples %f ' %(roc_auc_score(y_validation, predsValidation))
print 'ROC AUC Score on the validation examples %f ' %(roc_auc_score(y_validation, predsValidation))

ROC AUC Score on the training examples 0.569070 
ROC AUC Score on the validation examples 0.542704 


In [161]:
# full dataset
X_full = munger.X_full
y_full = munger.y_full

In [162]:
## fit on whole dataset
clf.fit(X_full, y_full)

Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=5,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [163]:
predictions = clf.predict_proba(X_test)[:, 1]

In [164]:
# submission dataframe
submission_df = pd.read_csv('../data/numerai_datasets/numerai_example_predictions.csv')

In [165]:
prepare_submission(submission_df, predictions, 'random_forest_classifier.csv')